In [1]:
from smartcard import ATR
from smartcard.System import readers

In [17]:
rarr=readers()
if len(rarr) == 0:
    print("No readers detected")
else:
    reader = rarr[0]
    print(reader)

Generic Smart Card Reader Interface


In [35]:
connection = reader.createConnection()
connection.connect()

In [36]:
atr=bytes(connection.getATR())
atr.hex()

'3bfe1800008031fe45803180664090a5102e1083019000f2'

In [37]:
def select(appletID):
    data = list(bytes.fromhex(appletID))
    # Select:
    # CLA = 0x00
    # INS = 0xA4
    # P1 = 0x04
    # P2 = 0x00
    # Data = the instance AID
    cmd = [0x00, # CLA
              0xA4, # INS
              0x04, # P1
              0x00, # P2
              len(data), # Lc (content length)
          ] + data + [0x00]
    data, *sw = connection.transmit(cmd)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [38]:
select("B00B5111CA01")

b''

In [39]:
def request(APDU):
    cmd = list(bytes.fromhex(APDU))
    data, *sw = connection.transmit(cmd)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [40]:
# get secret
request("B0A100000000")

b'I am a teapot gimme some tea plz'

In [31]:
secret = "omfg i am java dev now"
print(len(secret))
data = bytes([len(secret)])+secret.encode()
data.hex()

32


'204920616d206120746561706f742067696d6d6520736f6d652074656120706c7a'

In [30]:
# put secret
request("B0A20000"+data.hex()+"00")

b'omfg i am java dev now'

In [33]:
connection.disconnect()